<a href="https://colab.research.google.com/github/YUVALLEVI1997/BI-Project/blob/aviel/music_STTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install jsonpath_ng

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 6.4 MB/s eta 0:00:00


In [2]:
import os
import json
import numpy as np
import pandas as pd
import sqlite3
import functools as ft
import matplotlib.pyplot as plt
from abc import ABC, abstractmethod
from jsonpath_ng import parse
from enum import Enum
%matplotlib inline

## Data

In [3]:
original_file = pd.read_excel("musicData_ex.xlsx")
original_file
original_file .to_json("musicData_ex.json")
json_original = original_file .astype(str).to_dict(orient='records')
json_original

[{'Unnamed: 0': '0',
  'track_id': '5SuOikwiRyPMVoIQDJUgSV',
  'artists': 'Gen Hoshino',
  'album_name': 'Comedy',
  'track_name': 'Comedy',
  'popularity': '73',
  'duration_ms': '230666',
  'explicit': 'False',
  'danceability': '0.676',
  'energy': '0.461',
  'key': '1',
  'loudness': '-6.746',
  'mode': '0',
  'speechiness': '0.143',
  'acousticness': '0.0322',
  'instrumentalness': '1.01e-06',
  'liveness': '0.358',
  'valence': '0.715',
  'tempo': '87.917',
  'time_signature': '4',
  'track_genre': 'acoustic'},
 {'Unnamed: 0': '1',
  'track_id': '4qPNDBW1i3p13qLCt0Ki3A',
  'artists': 'Ben Woodward',
  'album_name': 'Ghost (Acoustic)',
  'track_name': 'Ghost - Acoustic',
  'popularity': '55',
  'duration_ms': '149610',
  'explicit': 'False',
  'danceability': '0.42',
  'energy': '0.166',
  'key': '1',
  'loudness': '-17.235',
  'mode': '1',
  'speechiness': '0.0763',
  'acousticness': '0.924',
  'instrumentalness': '5.56e-06',
  'liveness': '0.101',
  'valence': '0.267',
  'tempo'

###  Abstract Base Calss (ABC) for mutual methods

In [4]:
class Interface(ABC):
    @abstractmethod
    def get_data_by_field(self, field_name):
        """Fetch the data by given feild name """

    @abstractmethod
    def get_data_by_id(self, id):
        """Fetch the data by given ID  """

    @abstractmethod
    def get(self):
        """Fetch all data """

In [5]:
class get_data(Interface):
    def get_data_by_id(self, id):
        self.id = id
        data = self.get
        for x in data:
            if x.get("id").__str__() == self.id.__str__():
                return x
        return None

    def get_data_by_field(self, field_name):
        data = self.get
        for item in data:
            for key, value in item.items():
                if key == field_name:
                    return item
        return None


### Transform Operations
inherithed from Enum - class that automatic enumrate the variables

In [6]:
class TransformMask(Enum):
    # add here any masks you want
    CLEAN_STRING = ".strip().lower()"
    CAPITAL_LETTER = ".strip().lower().title()"
    NO_ACT = ""
    REPLACE_TO_BOOL = ""


    @property
    def replace_to_bool(num):
       if num>65:
          return "true"
       else:
          return "false"

### Database Class - Define Common Properties for Source, Target, Mapping

In [18]:
class Database:
    def __init__(self):
        self.db = {
            "source": [],
            "destination": [],
            "transform": [],
            "mapping": []
        }
        self.add_source(1,"popularity","$.popularity","str")
        self.add_source(2,"danceability","$.danceability","str")
        self.add_source(3,"track_genre","$.track_genre","str")

        self.add_destination(1,"popularity","int","0")
        self.add_destination(2,"danceability","float","0")
        self.add_destination(3,"genre","str","n/a")

        self.add_transform(1,'CLEAN_STRING')
        self.add_transform(2,'CAPITAL_LETTER')
        self.add_transform(3,'NO_ACT')

        self.add_mapping(1,1,1,3)
        self.add_mapping(2,2,2,3)
        self.add_mapping(3,3,3,2)

    def add_source(self,id,field,map,ty):
      if ty not in ["int","float","str"]:
        print("Error!")
      self.db["source"].append({  "id":id,
                                  "source_field_name":field,
                                  "source_field_mapping":map,
                                  "source_field_type":ty,
                                  "is_required": True,
      })

    def add_destination(self,id,field,ty,default):
        if ty not in ["int","float","str","Bool"]:
          print("Error destination.")
        self.db["destination"].append(
            {         "id": id,
                      "destination_field_name": field,
                      "destination_field_mapping": field,
                      "destination_field_type": ty,
                      "default_value": default,
                      "destination_table": "fact"
                  })

    def add_transform(self,id, transform_mask):
        self.db["transform"].append({
                      "id": id,
                      "transform_mask": transform_mask
                  })

    def add_mapping(self,id, mapping_source,mapping_destination,mapping_transform):
        self.db["mapping"].append({
                      "id": id,
                      "mapping_source": mapping_source,
                      "mapping_destination": mapping_destination,
                      "mapping_transform": mapping_transform,
                      "destination_table": "fact"
                  })

    @property
    def get_data_source_target_mapping(self):
        return self.db


### Source class , Target class , Transform Class , Mapping class

In [8]:
class Source(get_data, Database):
    def __init__(self):
        Database.__init__(self)

    @property
    def get(self):
        return self.get_data_source_target_mapping.get("source")

In [9]:
class Target(get_data, Database):
    def __init__(self):
        Database.__init__(self)

    @property
    def get(self):
        return self.get_data_source_target_mapping.get("destination")

In [10]:
class Transform(get_data, Database):

    def __init__(self):
        Database.__init__(self)

    @property
    def get(self):
        return self.get_data_source_target_mapping.get("transform", [])

In [11]:
class Mappings(get_data, Database):

    def __init__(self):
        Database.__init__(self)

    @property
    def get(self):
        return self.get_data_source_target_mapping.get("mapping")

    def get_data_by_field(self, field_name):
        return None

### Format Class - JSON

Search the source data value inside a JSON file

In [12]:
class JsonQuery:
    def __init__(self, json_path, json_data):
        self.json_path = json_path
        self.json_data = json_data

    def get(self):
        jsonpath_expression = parse(self.json_path)
        match = jsonpath_expression.find(self.json_data)
        source_data_value = match[0].value
        return source_data_value

### Combine it All - STTM

In [13]:
class STTM:
    def __init__(self, input_json):
        self.input_json = input_json
        self.mapping_instance = Mappings()
        self.source_instance = Source()
        self.destination_instance = Target()
        self.transform_instance = Transform()
        self.look_up_mask = {i.name: i.value for i in TransformMask}
        self.json_data_transformed = {}

    def _get_mapping_data(self):
        return self.mapping_instance.get

    def _get_mapping_source_data(self):
        return self.source_instance.get

    def get_transformed_data(self):

        for mappings in self._get_mapping_data():

            """fetch the source mapping """
            mapping_source_id = mappings.get("mapping_source")
            mapping_destination_id = mappings.get("mapping_destination")
            mapping_transform_id = mappings.get("mapping_transform")

            mapping_source_data = self.source_instance.get_data_by_id(id=mapping_source_id)
            transform_data = self.transform_instance.get_data_by_id(id=mapping_transform_id)

            """Fetch Source  field Name"""
            source_field_name = mapping_source_data.get("source_field_name")

            """if field given is not present incoming json """
            if source_field_name not in self.input_json.keys():
                if mapping_source_data.get("is_required"):
                    raise Exception(
                        "Alert ! Field {} is not present in JSON please FIX mappings ".format(source_field_name))
                else:
                    pass

            else:
                source_data_value = JsonQuery(
                    json_path=mapping_source_data.get("source_field_mapping"),
                    json_data=self.input_json
                ).get()

                """check the data type for source if matches with what we have """
                if mapping_source_data.get("source_field_type") != type(source_data_value).__name__:
                    if source_data_value is not None:
                        _message = (
                            "Alert ! Source Field :{} Datatype has changed from {} to {} ".format(source_field_name,
                                                                                                  mapping_source_data.get(
                                                                                                      "source_field_type"),
                                                                                                  type(
                                                                                                      source_data_value).__name__))
                        print(_message)
                        raise Exception(_message)

                """Query and fetch the Destination | target """
                destination_mappings_json_object = self.destination_instance.get_data_by_id(
                    id=mappings.get("mapping_destination"))

                destination_field_name = destination_mappings_json_object.get("destination_field_name")
                destination_field_type = destination_mappings_json_object.get("destination_field_type")

                dtypes = [str, float, list, int, set, dict]

                for dtype in dtypes:

                    """Datatype Conversion """
                    if destination_field_type == str(dtype.__name__):

                        """is source is none insert default value"""
                        if source_data_value is None:
                            self.json_data_transformed[destination_field_name] = dtype.__call__(
                                destination_mappings_json_object.get("default_value")
                            )

                        else:
                            """check if you have items to transform"""
                            if transform_data is not None:
                                """ check for invalid mask name """
                                if transform_data.get("transform_mask") not in list(self.look_up_mask.keys()):
                                    raise Exception(
                                        f"Specified Transform {transform_data.get('transform_mask')} is not available please select from following Options :{list(self.look_up_mask.keys())}")
                                else:
                                    mask_apply = self.look_up_mask.get(transform_data.get("transform_mask"))
                                    converted_dtype = dtype.__call__(source_data_value)
                                    mask = f'converted_dtype{mask_apply}'
                                    curated_value = eval(mask)
                                    self.json_data_transformed[destination_field_name] = curated_value

                            else:
                                self.json_data_transformed[destination_field_name] = dtype.__call__(source_data_value)

        return self.json_data_transformed

In [17]:
transformed_data = []
for item in json_original:
    helper = STTM(input_json=item)
    response = helper.get_transformed_data()
    transformed_data.append(response)
    print(response)

AttributeError: ignored

In [15]:
df = pd.DataFrame(transformed_data)
df

,popularity,danceability,genre
0,73,0.676,Acoustic
1,55,0.420,Acoustic
2,57,0.438,Acoustic
3,71,0.266,Acoustic
4,82,0.618,Acoustic
...,...,...,...
996,29,0.621,Acoustic
997,26,0.648,Acoustic
998,26,0.303,Acoustic
999,25,0.335,Acoustic
